# Install libraries

In [ ]:
%pip install scikit-learn
%pip install matplotlib
%pip install pandas
%pip install numpy
%pip install lightgbm
%pip install nbformat

# Load the dataset

In [2]:
import pandas as pd

# Load the dataset
heart_failure_df = pd.read_csv('../data/heart.csv')

feature_names = heart_failure_df.columns[:-1]
target_col_name = 'HeartDisease'
cat_features = ['Sex', 'ChestPainType', 'RestingECG', 'ExerciseAngina', 'ST_Slope', 'FastingBS']
num_features = ['Age', 'RestingBP', 'Cholesterol', 'MaxHR', 'Oldpeak']

assert len(feature_names) == len(cat_features) + len(num_features) # sanity check

heart_failure_df.head() # display the first few rows of the dataframe

,Age,Sex,ChestPainType,RestingBP,Cholesterol,FastingBS,RestingECG,MaxHR,ExerciseAngina,Oldpeak,ST_Slope,HeartDisease
0,40,M,ATA,140,289,0,Normal,172,N,0.0,Up,0
1,49,F,NAP,160,180,0,Normal,156,N,1.0,Flat,1
2,37,M,ATA,130,283,0,ST,98,N,0.0,Up,0
3,48,F,ASY,138,214,0,Normal,108,Y,1.5,Flat,1
4,54,M,NAP,150,195,0,Normal,122,N,0.0,Up,0


In [3]:
# Check data types
heart_failure_df.dtypes

Age                 int64
Sex                object
ChestPainType      object
RestingBP           int64
Cholesterol         int64
FastingBS           int64
RestingECG         object
MaxHR               int64
ExerciseAngina     object
Oldpeak           float64
ST_Slope           object
HeartDisease        int64
dtype: object

In [4]:
# FastingBS is a categorical feature, but it is encoded as an integer. We need to convert it to a boolean.
heart_failure_df['FastingBS'] = (heart_failure_df['FastingBS'] == 1).astype(bool)
heart_failure_df.head()

,Age,Sex,ChestPainType,RestingBP,Cholesterol,FastingBS,RestingECG,MaxHR,ExerciseAngina,Oldpeak,ST_Slope,HeartDisease
0,40,M,ATA,140,289,False,Normal,172,N,0.0,Up,0
1,49,F,NAP,160,180,False,Normal,156,N,1.0,Flat,1
2,37,M,ATA,130,283,False,ST,98,N,0.0,Up,0
3,48,F,ASY,138,214,False,Normal,108,Y,1.5,Flat,1
4,54,M,NAP,150,195,False,Normal,122,N,0.0,Up,0


In [5]:
# Check for missing values
missing_values = heart_failure_df.isnull().sum()
print(missing_values)

Age               0
Sex               0
ChestPainType     0
RestingBP         0
Cholesterol       0
FastingBS         0
RestingECG        0
MaxHR             0
ExerciseAngina    0
Oldpeak           0
ST_Slope          0
HeartDisease      0
dtype: int64


# Helper functions

In [ ]:
from matplotlib import pyplot as plt
import numpy as np
from sklearn.preprocessing import LabelEncoder # Encode categorical features


label_encoder = LabelEncoder()

def encode_categorical_features(df):
    for col in df.columns:
        if df[col].dtype == 'object' or df[col].dtype == 'bool' or df[col].dtype == 'category':
            df[col] = label_encoder.fit_transform(df[col])


def encode_categorical_features_as_frequency_encoding(df):
    for col in df.columns:
        if (df[col].dtype == 'object' or df[col].dtype == 'bool' or df[col].dtype == 'category'):
            df[col] = df[col].map(df[col].value_counts())


# Plot confusion matrix with annotations
def plot_confusion_matrix(cm, classes=['Negative', 'Positive'],
                          normalize=False,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        # print("Normalized confusion matrix")
    else:
        pass
        # print('Confusion matrix, without normalization')

    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    fmt = '.2f' if normalize else 'd'
    thresh = cm.max() / 2.
    for i in range(cm.shape[0]):
        for j in range(cm.shape[1]):
            plt.text(j, i, format(cm[i, j], fmt),
                     ha="center", va="center",
                     color="white" if cm[i, j] > thresh else "black")

    plt.ylabel('True label')
    plt.xlabel('Predicted label')
    plt.tight_layout()


def print_feature_importances(feature_importances, feature_names=feature_names):
    # Feature importances
    indices = np.argsort(feature_importances)[::-1]

    # Print the feature ranking
    print("Feature ranking:")
    for f in range(len(feature_names)):
        print("%d. feature %s (%f)" % (f + 1, feature_names[indices[f]], feature_importances[indices[f]]))